In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from IPython.display import display
import tensorflow as tf
from sklearn.utils import resample

In [2]:
rental_dataset = pd.read_csv("https://raw.githubusercontent.com/Capstone-Rental-Marketplace/SewaIn/main/Dataset/rental_dataset.csv")
rental_dataset

,id_user,id_rental,id_vehicle,rating,sum_day_of_rental,total_pay
0,162,43,MTR009,5,4,320000
1,1527,62,MTR0020,5,3,270000
2,10989,7,MBL0025,5,2,1600000
3,5908,119,MBL0026,5,2,800000
4,7730,80,MTR0010,5,4,1000000
...,...,...,...,...,...,...
14590,13880,91,MBL009,5,4,1600000
14591,2547,111,MTR003,4,3,390000
14592,6385,92,MTR0013,5,3,720000
14593,10382,25,MBL009,5,4,2000000


In [3]:
rental_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14595 entries, 0 to 14594
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id_user            14595 non-null  int64 
 1   id_rental          14595 non-null  int64 
 2   id_vehicle         14595 non-null  object
 3   rating             14595 non-null  int64 
 4   sum_day_of_rental  14595 non-null  int64 
 5   total_pay          14595 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 684.3+ KB


In [4]:
len(rental_dataset['id_user'].unique())

13895

In [5]:
rental_dataset['rating'].value_counts()


rating
5    9825
4    4553
3     146
2      45
1      26
Name: count, dtype: int64

In [6]:
user_dataset = pd.read_csv("https://raw.githubusercontent.com/Capstone-Rental-Marketplace/SewaIn/main/Dataset/user_dataset.csv")
user_dataset

,id_user,user_name,user_email
0,1,Abigail Davis,obriendaniel@gmail.com
1,2,Jason Wall,rothbryan@gmail.com
2,3,Trent Sandoval,trent.sandoval@gmail.com
3,4,Damaris Oneal,damaris.oneal@gmail.com
4,5,Kristin Morse,umorrison@gmail.com
...,...,...,...
13890,13891,muhammad andriansyah damanik,muhammadandriansyahdamanik@gmail.com
13891,13892,Susan Stewart,masontracy@gmail.com
13892,13893,Bria Spence,bria.spence@gmail.com
13893,13894,Elle Spears,elle.spears@gmail.com


In [7]:
shop_dataset = pd.read_csv("https://raw.githubusercontent.com/Capstone-Rental-Marketplace/SewaIn/main/Dataset/shop_dataset.csv")
shop_dataset

,id_rental,rental_name,vehicle_type,alamat,kabupaten/kota,kecamatan,no_hp,rating
0,1,My Bike Rental Bali (PT Trans Mina Mulia),Motor,"Jl. Raya Kaba-Kaba, Kaba-Kaba, Kec. Kediri, Ka...",Kabupaten Tabanan,Kec. Kediri,8.214656e+10,5
1,2,Rent Car Marga Dewata,Mobil,"Jl. Kebo Iwa Bongan No.19, Bongan, Puseh, Kabu...",Kabupaten Tabanan,Kec. Puseh,8.786550e+10,4.9
2,3,Trea Trans,Mobil,"Jl. Kresna 3 banjar pangkung No.25, Delod Peke...",Kabupaten Tabanan,Kec. Tabanan,8.898703e+10,5
3,4,Agas Motorbike Rental,Motor,"Pandak Gede, Kec. Kediri, Kabupaten Tabanan, B...",Kabupaten Tabanan,Kec. Kediri,8.579229e+10,5
4,5,HARYS Rent Car & Tour,Mobil,"Perumahan Graha Luhur Damai Blok Q.29, Pandak ...",Kabupaten Tabanan,Kec. Kediri,8.786556e+10,4.4
...,...,...,...,...,...,...,...,...
116,116,3 Monkeys Rental Bali,Mobil & Motor,"Jl. Kubu Gn. No.11, Dalung, Kec. Kuta Utara, K...",Kabupaten Badung,Kec. Kuta Utara,8.589999e+10,5
117,117,Persewaan Motor & Mobil( Duta Rent ),Mobil & Motor,"CFP2+WMQ, Jl. Pantai, Kp. Kusamba, Kec. Dawan,...",Kabupaten Klukung,Kec. Dawan,8.123458e+10,5
118,118,Rental scooter & car bali utara murah,Mobil,"Jl. Tunjung No.42, Kaliasem, Kec. Banjar, Kabu...",Kabupaten Buleleng,Kec. Banjar,8.133856e+10,5
119,119,"Scooter Rental, Bike Rental, Car Rental (Bangk...",Mobil & Motor,"Jl. Hanoman No.30, Ubud, Kecamatan Ubud, Kabup...",Kabupaten Gianyar,Kec. Ubud,8.234099e+10,4.7


In [8]:
shop_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_rental       121 non-null    int64  
 1   rental_name     121 non-null    object 
 2   vehicle_type    121 non-null    object 
 3   alamat          121 non-null    object 
 4   kabupaten/kota  121 non-null    object 
 5   kecamatan       121 non-null    object 
 6   no_hp           121 non-null    float64
 7   rating          121 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 7.7+ KB


In [9]:
shop_dataset = shop_dataset.drop(columns={'rating', 'vehicle_type'})

In [10]:
vehicle_dataset = pd.read_csv("https://raw.githubusercontent.com/Capstone-Rental-Marketplace/SewaIn/main/Dataset/vehicle_dataset.csv")
vehicle_dataset

,id_vehicle,vehicle_name,engine_type,power,torque,colors,fuel_capacity,transmission,vehicle_type,number_of_seats,brand,model
0,MBL001,Suzuki Ertiga,1.5L VVT Petrol,103 hp,138 Nm,Black,45 liters,5-speed Manual / 4-speed Automatic,Mobil,7,Suzuki,Ertiga
1,MBL002,Suzuki Ertiga,1.5L VVT Petrol,103 hp,138 Nm,Silver,45 liters,5-speed Manual / 4-speed Automatic,Mobil,7,Suzuki,Ertiga
2,MBL003,Suzuki Ertiga,1.5L VVT Petrol,103 hp,138 Nm,White,45 liters,5-speed Manual / 4-speed Automatic,Mobil,7,Suzuki,Ertiga
3,MBL004,Honda Mobilio,1.5L i-VTEC Petrol,118 hp,145 Nm,Black,42 liters,6-speed Manual / CVT,Mobil,7,Honda,Mobilio
4,MBL005,Honda Mobilio,1.5L i-VTEC Petrol,118 hp,145 Nm,White,42 liters,6-speed Manual / CVT,Mobil,7,Honda,Mobilio
...,...,...,...,...,...,...,...,...,...,...,...,...
76,MTR0035,Yamaha NMAX,"Liquid-cooled, 4-stroke",15.1 hp,13.9 Nm,Black,7.1 liters,CVT,Motor,2,Yamaha,NMAX
77,MTR0036,Yamaha NMAX,"Liquid-cooled, 4-stroke",15.1 hp,13.9 Nm,Grey,7.1 liters,CVT,Motor,2,Yamaha,NMAX
78,MTR0037,Yamaha MT-25,"Liquid-cooled, 4-stroke",35.5 hp,23.6 Nm,White,14 liters,6-speed manual,Motor,2,Yamaha,MT-25
79,MTR0038,Yamaha MT-25,"Liquid-cooled, 4-stroke",35.5 hp,23.6 Nm,Black,14 liters,6-speed manual,Motor,2,Yamaha,MT-25


In [11]:
vehicle_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_vehicle       81 non-null     object
 1   vehicle_name     81 non-null     object
 2   engine_type      81 non-null     object
 3   power            81 non-null     object
 4   torque           81 non-null     object
 5   colors           81 non-null     object
 6   fuel_capacity    81 non-null     object
 7   transmission     81 non-null     object
 8   vehicle_type     81 non-null     object
 9   number_of_seats  81 non-null     int64 
 10  brand            81 non-null     object
 11  model            81 non-null     object
dtypes: int64(1), object(11)
memory usage: 7.7+ KB


In [12]:
ownership_dataset = pd.read_csv("https://raw.githubusercontent.com/Capstone-Rental-Marketplace/SewaIn/main/Dataset/ownership_rental_vehicle_dataset.csv")
ownership_dataset

,id_rental,id_vehicle,price_per_day,num_vehicle
0,58,MBL004,500000,3
1,9,MTR0032,100000,3
2,68,MBL0020,500000,3
3,54,MTR0018,110000,3
4,28,MBL0024,450000,4
...,...,...,...,...
4918,10,MTR0017,110000,3
4919,88,MBL004,350000,3
4920,104,MBL0024,450000,4
4921,97,MBL0032,800000,3


In [13]:
ownership_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4923 entries, 0 to 4922
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_rental      4923 non-null   int64 
 1   id_vehicle     4923 non-null   object
 2   price_per_day  4923 non-null   int64 
 3   num_vehicle    4923 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 154.0+ KB


In [60]:

# Join data
data = rental_dataset.merge(ownership_dataset, on=['id_rental', 'id_vehicle'])
data = data.merge(vehicle_dataset, on='id_vehicle')
data = data.merge(shop_dataset, on='id_rental')
data = data.merge(user_dataset, on='id_user')



In [61]:
# Convert all columns to string
data['price_per_day'] = data['price_per_day'].astype(str)
data['num_vehicle'] = data['num_vehicle'].astype(str)
data['engine_type'] = data['engine_type'].astype(str)
data['power'] = data['power'].astype(str)
data['torque'] = data['torque'].astype(str)
data['colors'] = data['colors'].astype(str)
data['fuel_capacity'] = data['fuel_capacity'].astype(str)
data['transmission'] = data['transmission'].astype(str)
data['vehicle_type'] = data['vehicle_type'].astype(str)
data['number_of_seats'] = data['number_of_seats'].astype(str)
data['brand'] = data['brand'].astype(str)
data['model'] = data['model'].astype(str)
data['rental_name'] = data['rental_name'].astype(str)

# Combine the features into a single string column
#data['data_features'] = data['price_per_day'] + " " + data['engine_type'] + " " + data['power'] + " " + data['torque'] + " " + data['colors'] + " " + data['fuel_capacity'] + " " + data['transmission'] + " " + data['vehicle_type'] + " " + data['number_of_seats'] + " " + data['brand'] + " " + data['model'] + " " + data["rental_name"]
data['data_features'] = data['colors'] + " " + data['vehicle_type'] + " " + data['number_of_seats'] + " " + data['brand'] + " " + data['model'] + " " + data['price_per_day']
data = data[['id_user', 'id_rental', 'id_vehicle', 'rating', "data_features"]]
data

,id_user,id_rental,id_vehicle,rating,data_features
0,162,43,MTR009,5,Black Motor 2 Honda Beat 80000
1,2596,43,MTR009,5,Black Motor 2 Honda Beat 80000
2,12633,43,MTR009,4,Black Motor 2 Honda Beat 80000
3,12436,43,MTR009,4,Black Motor 2 Honda Beat 80000
4,3425,43,MTR009,5,Black Motor 2 Honda Beat 80000
...,...,...,...,...,...
13799,11659,117,MBL007,4,Black Mobil 7 Honda BR-V 600000
13800,7185,117,MBL0028,4,Red Mobil 7 Daihatsu Terios 450000
13801,9525,117,MBL0029,5,White Mobil 7 Daihatsu Terios 500000
13802,6365,117,MBL0020,5,White Mobil 7 Toyota Avanza 350000


In [62]:
#data['data_features'] = data['data_features'].fillna('')
data['data_features'] = data['data_features'].str.split(' ')
# Remove duplicates
#data = data.drop_duplicates(subset='data_features')
#data['data_features'] = data['data_features'].fillna('')
# Apply one-hot encoding

data_features = data['data_features'].apply(lambda x: pd.Series(1, index=x)).fillna(0).astype(int)

df_ratings_encode= pd.concat([data, data_features], axis=1)
df_ratings_encode.drop(columns=['data_features'],inplace=True)
df_ratings_encode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13804 entries, 0 to 13803
Columns: 104 entries, id_user to 950000
dtypes: int64(103), object(1)
memory usage: 11.0+ MB


In [59]:
df_ratings_encode

,id_user,id_rental,id_vehicle,rating,Black,Motor,2,Honda,Beat,Mobil,...,Sport,Rush,Mobilio,5,Brio,Kijang,Xpander,Terios,Suzuki,Ertiga
0,162,43,MTR009,5,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2596,43,MTR009,5,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,12633,43,MTR009,4,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,12436,43,MTR009,4,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,3425,43,MTR009,5,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13799,11659,117,MBL007,4,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
13800,7185,117,MBL0028,4,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
13801,9525,117,MBL0029,5,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
13802,6365,117,MBL0020,5,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
